In [1]:
import prepare_data
import pandas as pd

In [2]:
id2label, label2id, submission_file, test_ids_final, test_ocr_clean, train_ids_final, train_labels_final, train_set_ocr = prepare_data.read_files()

In [102]:
train_data = pd.read_csv('train_data_modified.csv')
test_data = pd.read_csv('test_data_modified.csv')

photos_metrics = pd.read_csv('train_metrics.csv')


In [103]:
train_data = train_data[train_data['isPolish'] == 0]
test_data = test_data[test_data['isPolish'] == 0]

## Data Preparation

In [5]:
train_data

,Unnamed: 0.1,Unnamed: 0,File,Text ING,ID,Text Adam,Nrow,Text Adam pl,isPolish,isEnglish,isOther
1490,1490,1710,570492ae-1a40-40cf-8d45-ed3591ea74ae,"nazwisko): NIP; I przez miasta, inazwisko)H?U ...",20,( Sptelipeyiię zako a o boga | | MISONE 2 A LN...,20,( Sptelipeyiie zako a o boga | | MISONE 2 A LN...,0,0,1
1650,1650,3596,1f8d02ff-2739-4d5b-8029-3d01700acb2e,contributors MARK MANN's Potograrhs have THOMA...,0,contribułors - j MARIK MANU słetegahs hoc THOM...,51,contribulors - j MARIK MANU sletegahs hoc THOM...,0,1,0
1651,1651,8555,7b3ecce6-cea7-4378-af81-dab9434e8ca7,thing amnardiLmythat sj Sta adnoreanimaled ho ...,0,00000 RL W O OPR PA e thiagwarm aid ferry that...,122,00000 RL W O OPR PA e thiagwarm aid ferry that...,0,1,0
1652,1652,4794,2ac406e6-2fcd-45c1-989d-ee17fca95425,Catalina Marketing 365 West Passaic Strect Roc...,0,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",25,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",0,1,0
1653,1653,10120,78a696be-d7b7-4cb7-a37f-ee3587f601e2,IMAGE NOT AVAILABLE ONLINE The material refere...,0,IMAGE NOT AVAILABLE ONLINE The material refere...,9,IMAGE NOT AVAILABLE ONLINE The material refere...,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9750,9750,7098,6e914249-5314-46c6-bf95-f30228fb2b83,To: Weber Patti Itterman From: Jackie Klecan D...,7,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",12,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",0,1,0
9751,9751,7042,d3d18ddd-a110-443a-aa82-20b7864ba3b8,ZVSA Basehall IODAY Weexly Koith Cutler Publi...,7,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",28,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",0,1,0
9752,9752,4303,2061e531-a091-4f40-937f-a7fa8bada187,"PHILIP MORRIS US.4. ICD PAAK AVENUE New York, ...",7,"A 41% KEOMAS ć 9 y ST USA. 0 PARK AVENUE, NEM ...",39,"A 41% KEOMAS c 9 y ST USA. 0 PARK AVENUE, NEM ...",0,1,0
9753,9753,4455,39b58825-ae30-4376-b332-611b14d88d03,0ba Sil Eus Ovx 543 1052 M~ chro )50 352 M Lw;...,7,"EP A — O o 7 p"" pzzą 0. Te fo dzi: 843 1082 e”...",47,"EP A — O o 7 p"" pzza 0. Te fo dzi: 843 1082 e”...",0,1,0


In [104]:
df = train_data.drop(columns=['isPolish','Unnamed: 0','isEnglish', 'Text Adam'])
df = df.drop(columns=["Unnamed: 0.1"])

df_val = test_data.drop(columns=['isPolish','Unnamed: 0','isEnglish', 'Text Adam'])
df_val = df_val.drop(columns=["Unnamed: 0.1"])
df

,File,Text ING,ID,Nrow,Text Adam pl,isOther
1490,570492ae-1a40-40cf-8d45-ed3591ea74ae,"nazwisko): NIP; I przez miasta, inazwisko)H?U ...",20,20,( Sptelipeyiie zako a o boga | | MISONE 2 A LN...,1
1650,1f8d02ff-2739-4d5b-8029-3d01700acb2e,contributors MARK MANN's Potograrhs have THOMA...,0,51,contribulors - j MARIK MANU sletegahs hoc THOM...,0
1651,7b3ecce6-cea7-4378-af81-dab9434e8ca7,thing amnardiLmythat sj Sta adnoreanimaled ho ...,0,122,00000 RL W O OPR PA e thiagwarm aid ferry that...,0
1652,2ac406e6-2fcd-45c1-989d-ee17fca95425,Catalina Marketing 365 West Passaic Strect Roc...,0,25,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",0
1653,78a696be-d7b7-4cb7-a37f-ee3587f601e2,IMAGE NOT AVAILABLE ONLINE The material refere...,0,9,IMAGE NOT AVAILABLE ONLINE The material refere...,0
...,...,...,...,...,...,...
9750,6e914249-5314-46c6-bf95-f30228fb2b83,To: Weber Patti Itterman From: Jackie Klecan D...,7,12,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",0
9751,d3d18ddd-a110-443a-aa82-20b7864ba3b8,ZVSA Basehall IODAY Weexly Koith Cutler Publi...,7,28,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",0
9752,2061e531-a091-4f40-937f-a7fa8bada187,"PHILIP MORRIS US.4. ICD PAAK AVENUE New York, ...",7,39,"A 41% KEOMAS c 9 y ST USA. 0 PARK AVENUE, NEM ...",0
9753,39b58825-ae30-4376-b332-611b14d88d03,0ba Sil Eus Ovx 543 1052 M~ chro )50 352 M Lw;...,7,47,"EP A — O o 7 p"" pzza 0. Te fo dzi: 843 1082 e”...",0


In [105]:
import re
from fuzzywuzzy import fuzz

string_column = "Text Adam pl"
df[string_column] = df[string_column].astype(str)
df['string_length'] = df[string_column].apply(lambda x: len(x))

# Add a column for the count of capital letters
df['capital_letters_count'] = df[string_column].apply(lambda x: sum(1 for c in x if c.isupper()))

# Add a column for the count of numbers
df['numbers_count'] = df[string_column].apply(lambda x: sum(1 for c in x if c.isdigit()))
# Add a column for the count of question marks
df['question_marks_count'] = df[string_column].apply(lambda x: x.count('?'))
df['exclamation_marks_count'] = df[string_column].apply(lambda x: x.count('!'))
# Add a column for the count of currency signs
df['currency_signs_count'] = df[string_column].apply(lambda x: x.count('$') + x.count('€') + x.count('£'))


# american phone regex
pattern = r"\b\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"

# american address regex
pattern1 = r"^\d+\s+([a-zA-Z]+\s+)+[a-zA-Z]+,\s*[A-Za-z]{2}\s*\d{5}(?:-\d{4})?$"

# Add a column for the flag indicating pattern match
df['flag_phone'] = df[string_column].apply(lambda x: 1 if re.search(pattern, x) else 0)

# Define the pattern to search for
pattern = r"\bi want it\b"

# Add a column for the flag indicating pattern match
df['flag_text'] = df[string_column].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)





df_val[string_column] = df_val[string_column].astype(str)
df_val['string_length'] = df_val[string_column].apply(lambda x: len(x))

# Add a column for the count of capital letters
df_val['capital_letters_count'] = df_val[string_column].apply(lambda x: sum(1 for c in x if c.isupper()))

# Add a column for the count of numbers
df_val['numbers_count'] = df_val[string_column].apply(lambda x: sum(1 for c in x if c.isdigit()))
# Add a column for the count of question marks
df_val['question_marks_count'] = df_val[string_column].apply(lambda x: x.count('?'))
df_val['exclamation_marks_count'] = df_val[string_column].apply(lambda x: x.count('!'))
# Add a column for the count of currency signs
df_val['currency_signs_count'] = df_val[string_column].apply(lambda x: x.count('$') + x.count('€') + x.count('£'))


# american phone regex
pattern = r"\b\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"

# american address regex
pattern1 = r"^\d+\s+([a-zA-Z]+\s+)+[a-zA-Z]+,\s*[A-Za-z]{2}\s*\d{5}(?:-\d{4})?$"

# Add a column for the flag indicating pattern match
df_val['flag_phone'] = df_val[string_column].apply(lambda x: 1 if re.search(pattern, x) else 0)

# Define the pattern to search for
pattern = r"\bi want it\b"

# Add a column for the flag indicating pattern match
df_val['flag_text'] = df_val[string_column].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)

df

,File,Text ING,ID,Nrow,Text Adam pl,isOther,string_length,capital_letters_count,numbers_count,question_marks_count,exclamation_marks_count,currency_signs_count,flag_phone,flag_text
1490,570492ae-1a40-40cf-8d45-ed3591ea74ae,"nazwisko): NIP; I przez miasta, inazwisko)H?U ...",20,20,( Sptelipeyiie zako a o boga | | MISONE 2 A LN...,1,434,84,20,0,0,0,0,0
1650,1f8d02ff-2739-4d5b-8029-3d01700acb2e,contributors MARK MANN's Potograrhs have THOMA...,0,51,contribulors - j MARIK MANU sletegahs hoc THOM...,0,1723,463,18,1,0,0,0,0
1651,7b3ecce6-cea7-4378-af81-dab9434e8ca7,thing amnardiLmythat sj Sta adnoreanimaled ho ...,0,122,00000 RL W O OPR PA e thiagwarm aid ferry that...,0,2245,317,37,1,0,0,0,0
1652,2ac406e6-2fcd-45c1-989d-ee17fca95425,Catalina Marketing 365 West Passaic Strect Roc...,0,25,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",0,488,176,29,0,0,0,0,0
1653,78a696be-d7b7-4cb7-a37f-ee3587f601e2,IMAGE NOT AVAILABLE ONLINE The material refere...,0,9,IMAGE NOT AVAILABLE ONLINE The material refere...,0,231,30,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9750,6e914249-5314-46c6-bf95-f30228fb2b83,To: Weber Patti Itterman From: Jackie Klecan D...,7,12,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",0,302,18,16,0,0,0,0,0
9751,d3d18ddd-a110-443a-aa82-20b7864ba3b8,ZVSA Basehall IODAY Weexly Koith Cutler Publi...,7,28,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",0,846,59,22,0,0,0,0,0
9752,2061e531-a091-4f40-937f-a7fa8bada187,"PHILIP MORRIS US.4. ICD PAAK AVENUE New York, ...",7,39,"A 41% KEOMAS c 9 y ST USA. 0 PARK AVENUE, NEM ...",0,1259,234,63,0,0,3,1,0
9753,39b58825-ae30-4376-b332-611b14d88d03,0ba Sil Eus Ovx 543 1052 M~ chro )50 352 M Lw;...,7,47,"EP A — O o 7 p"" pzza 0. Te fo dzi: 843 1082 e”...",0,2069,257,39,1,1,1,0,0


In [265]:
import nltk
#import IProgress
import ipywidgets
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
tqdm.pandas()

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wkosi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wkosi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wkosi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\wkosi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [266]:
def preprocess_text(text):
    # Tokenize the text
    #tokens = word_tokenize(str(text).lower())
    tokens = str(text).lower().split()
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Join the tokens back into a string
    #preprocessed_text = ' '.join(tokens)
    return tokens

In [106]:
email_keywords = ["original message", "cc:", "subject:", "re:", "attachments"]
letter_keywords = ["dear", "regards", "sincerely", "miss", "sir"]
invoices_keywords = ["invoice", "voucher", "amount","description","estimate","rate","payment","price"]
forms_keywords = ["form","request","report","confidential"]
memos_keywords = ["memo","memorandum"]
budget_keywords = [
    "budget", "period", "cost", "expenditure", "subtotal", "total"
]
specification_keywords = ["specification", "spec"]
scientific_report_keywords = ["abstract", "introduction", "method", "results","curriculum vitae","biographical sketch","redacted material"]
advertisment_keywords = ["smoking",  "cigarettes"]




all_keywords = email_keywords + letter_keywords + invoices_keywords + forms_keywords + memos_keywords + budget_keywords + specification_keywords + scientific_report_keywords + advertisment_keywords
keywords_counter = dict(zip(all_keywords, [[] for i in range(len(all_keywords))]))

In [181]:
from collections import Counter
counter =0
for index, row in df.iterrows():
    for keyword in all_keywords:
        if re.findall(keyword,row['Text Adam pl'].lower()):
            keywords_counter[keyword].append(row['ID'])

for key, value in keywords_counter.items():
    print(f"Key: {key}, {Counter(value)}")



Key: original message, Counter({2: 11, 4: 8})
Key: cc:, Counter({2: 60, 8: 29, 7: 12, 17: 2, 4: 2, 9: 2, 5: 1, 15: 1, 1: 1, 16: 1})
Key: subject:, Counter({2: 225, 8: 151, 7: 62, 4: 30, 14: 3, 13: 3, 16: 2, 9: 1, 1: 1, 12: 1})
Key: re:, Counter({2: 97, 8: 79, 7: 70, 17: 52, 4: 37, 13: 31, 16: 26, 15: 19, 9: 17, 6: 15, 1: 13, 0: 12, 14: 11, 12: 10, 3: 5, 5: 4})
Key: attachments, Counter({2: 19, 8: 6, 4: 3, 16: 1, 7: 1})
Key: dear, Counter({7: 230, 13: 20, 5: 17, 4: 14, 15: 12, 2: 10, 9: 10, 1: 7, 14: 5, 8: 5, 0: 1, 17: 1, 12: 1})
Key: regards, Counter({7: 23, 8: 11, 2: 10, 4: 9, 15: 2, 6: 1, 16: 1})
Key: sincerely, Counter({7: 85, 4: 5, 5: 2, 13: 2, 8: 1, 12: 1})
Key: miss, Counter({4: 78, 7: 41, 14: 39, 12: 39, 9: 33, 1: 18, 2: 16, 6: 16, 15: 15, 8: 10, 16: 10, 13: 10, 0: 5, 17: 5, 5: 1})
Key: sir, Counter({15: 60, 9: 32, 7: 26, 14: 23, 8: 22, 4: 17, 1: 14, 16: 12, 12: 12, 6: 11, 13: 11, 0: 8, 17: 8, 5: 5, 2: 5, 3: 2})
Key: invoice, Counter({6: 106, 1: 22, 4: 7, 8: 3, 7: 3, 2: 1})
Key:

## Model

In [107]:
df

,File,Text ING,ID,Nrow,Text Adam pl,isOther,string_length,capital_letters_count,numbers_count,question_marks_count,exclamation_marks_count,currency_signs_count,flag_phone,flag_text
1490,570492ae-1a40-40cf-8d45-ed3591ea74ae,"nazwisko): NIP; I przez miasta, inazwisko)H?U ...",20,20,( Sptelipeyiie zako a o boga | | MISONE 2 A LN...,1,434,84,20,0,0,0,0,0
1650,1f8d02ff-2739-4d5b-8029-3d01700acb2e,contributors MARK MANN's Potograrhs have THOMA...,0,51,contribulors - j MARIK MANU sletegahs hoc THOM...,0,1723,463,18,1,0,0,0,0
1651,7b3ecce6-cea7-4378-af81-dab9434e8ca7,thing amnardiLmythat sj Sta adnoreanimaled ho ...,0,122,00000 RL W O OPR PA e thiagwarm aid ferry that...,0,2245,317,37,1,0,0,0,0
1652,2ac406e6-2fcd-45c1-989d-ee17fca95425,Catalina Marketing 365 West Passaic Strect Roc...,0,25,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",0,488,176,29,0,0,0,0,0
1653,78a696be-d7b7-4cb7-a37f-ee3587f601e2,IMAGE NOT AVAILABLE ONLINE The material refere...,0,9,IMAGE NOT AVAILABLE ONLINE The material refere...,0,231,30,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9750,6e914249-5314-46c6-bf95-f30228fb2b83,To: Weber Patti Itterman From: Jackie Klecan D...,7,12,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",0,302,18,16,0,0,0,0,0
9751,d3d18ddd-a110-443a-aa82-20b7864ba3b8,ZVSA Basehall IODAY Weexly Koith Cutler Publi...,7,28,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",0,846,59,22,0,0,0,0,0
9752,2061e531-a091-4f40-937f-a7fa8bada187,"PHILIP MORRIS US.4. ICD PAAK AVENUE New York, ...",7,39,"A 41% KEOMAS c 9 y ST USA. 0 PARK AVENUE, NEM ...",0,1259,234,63,0,0,3,1,0
9753,39b58825-ae30-4376-b332-611b14d88d03,0ba Sil Eus Ovx 543 1052 M~ chro )50 352 M Lw;...,7,47,"EP A — O o 7 p"" pzza 0. Te fo dzi: 843 1082 e”...",0,2069,257,39,1,1,1,0,0


In [108]:
df2 = df[~df.ID.isin([10,11,18,19,20])]
df2_val = df_val[~df_val.ID.isin([10,11,18,19,20])]

In [109]:
for keyword in all_keywords:
    df2[keyword] = 0
    df2_val[keyword] = 0

for index, row in df2.iterrows():
    for keyword in all_keywords:
        df2.at[index, keyword] += len(re.findall(keyword,row[string_column].lower()))
        #df2.at[index, keyword] = 1 if fuzz.partial_ratio(keyword, row["Text Adam pl"].lower()) > 90 else 0

for index, row in df2_val.iterrows():
    for keyword in all_keywords:
        df2_val.at[index, keyword] += len(re.findall(keyword,row[string_column].lower()))

C:\Users\szymo\AppData\Local\Temp\ipykernel_17440\111727474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[keyword] = 0
C:\Users\szymo\AppData\Local\Temp\ipykernel_17440\111727474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[keyword] = 0
C:\Users\szymo\AppData\Local\Temp\ipykernel_17440\111727474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [17]:
df2

,File,Text ING,ID,Nrow,Text Adam pl,isOther,string_length,capital_letters_count,numbers_count,question_marks_count,...,spec,abstract,introduction,method,results,curriculum vitae,biographical sketch,redacted material,smoking,cigarettes
1650,1f8d02ff-2739-4d5b-8029-3d01700acb2e,contributors MARK MANN's Potograrhs have THOMA...,0,51,contribulors - j MARIK MANU sletegahs hoc THOM...,0,1723,463,18,1,...,0,0,0,0,0,0,0,0,1,0
1651,7b3ecce6-cea7-4378-af81-dab9434e8ca7,thing amnardiLmythat sj Sta adnoreanimaled ho ...,0,122,00000 RL W O OPR PA e thiagwarm aid ferry that...,0,2245,317,37,1,...,0,0,0,0,0,0,0,0,0,0
1652,2ac406e6-2fcd-45c1-989d-ee17fca95425,Catalina Marketing 365 West Passaic Strect Roc...,0,25,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",0,488,176,29,0,...,0,0,0,0,0,0,0,0,0,0
1653,78a696be-d7b7-4cb7-a37f-ee3587f601e2,IMAGE NOT AVAILABLE ONLINE The material refere...,0,9,IMAGE NOT AVAILABLE ONLINE The material refere...,0,231,30,0,0,...,0,0,0,0,0,0,0,0,0,0
1654,1f3fbdca-c2be-44c9-b788-eed483a8e9ae,Currently sold only surgeon GeNEAal'S WarNing ...,0,31,SURGEON GENENAL'S WARNING; Smoking Currently s...,0,699,234,7,0,...,0,0,0,0,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9750,6e914249-5314-46c6-bf95-f30228fb2b83,To: Weber Patti Itterman From: Jackie Klecan D...,7,12,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",0,302,18,16,0,...,0,0,0,0,0,0,0,0,0,0
9751,d3d18ddd-a110-443a-aa82-20b7864ba3b8,ZVSA Basehall IODAY Weexly Koith Cutler Publi...,7,28,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",0,846,59,22,0,...,0,0,0,0,0,0,0,0,0,0
9752,2061e531-a091-4f40-937f-a7fa8bada187,"PHILIP MORRIS US.4. ICD PAAK AVENUE New York, ...",7,39,"A 41% KEOMAS c 9 y ST USA. 0 PARK AVENUE, NEM ...",0,1259,234,63,0,...,0,0,0,0,0,0,0,0,0,0
9753,39b58825-ae30-4376-b332-611b14d88d03,0ba Sil Eus Ovx 543 1052 M~ chro )50 352 M Lw;...,7,47,"EP A — O o 7 p"" pzza 0. Te fo dzi: 843 1082 e”...",0,2069,257,39,1,...,1,0,0,0,0,0,0,0,0,0


In [110]:
mapping = dict(zip([ 0,  1,  2,  3,  4 , 5,  6,  7,  8 , 9 ,12, 13, 14, 15, 16 ,17],[ 0,  1 , 2,  3 , 4 , 5 , 6 , 7,  8,  9 ,10 ,11 ,12, 13, 14 ,15] ))

In [111]:
df2["ID"]=df2["ID"].map(mapping).astype(int)
df2_val["ID"]=df2_val["ID"].map(mapping).astype(int)

C:\Users\szymo\AppData\Local\Temp\ipykernel_17440\2915549792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["ID"]=df2["ID"].map(mapping).astype(int)


In [112]:
df2["capital_letters_count"] = df2["capital_letters_count"] / (df2["string_length"]+1)
df2_val["capital_letters_count"] = df2_val["capital_letters_count"] / (df2_val["string_length"]+1)

C:\Users\szymo\AppData\Local\Temp\ipykernel_17440\1959765153.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["capital_letters_count"] = df2["capital_letters_count"] / (df2["string_length"]+1)


In [163]:
print(df2.shape)
print(df2_val.shape)

(8075, 55)
(907, 55)


In [156]:
import pandas as pd
import glob

# Get a list of CSV file paths
file_paths = glob.glob('HackING_mISIe_full_train/*.csv')  # Replace 'path/to/csv/files/' with your actual file path

# Initialize an empty list to store the DataFrames
dfs_metr = []

# Read and concatenate the CSV files
for file_path in file_paths:
    df_metr = pd.read_csv(file_path)
    dfs_metr.append(df_metr)

# Concatenate the DataFrames into a single DataFrame
combined_df_metr = pd.concat(dfs_metr, ignore_index=True)

# Print the combined DataFrame
combined_df_metr

,Unnamed: 0,filepath,numberEmpty,nonWhiteFraction,possibleShapes,possibleImages,nonEmptySections
0,0,./DANE/train_set/advertisement\00133f89-017d-4...,0.146341,0.575719,4,2,0.500000
1,1,./DANE/train_set/advertisement\00a3e900-6616-4...,0.161616,0.722688,2,1,0.500000
2,2,./DANE/train_set/advertisement\0116aef3-5834-4...,0.210526,0.086930,5,5,0.333333
3,3,./DANE/train_set/advertisement\024b107d-cd7f-4...,0.133333,0.814986,1,1,0.500000
4,4,./DANE/train_set/advertisement\0296a9e1-5bec-4...,0.142857,0.324584,2,2,0.500000
...,...,...,...,...,...,...,...
10844,195,./DANE/train_set/umowa_na_odleglosc_odstapieni...,0.187500,0.017731,0,0,0.500000
10845,196,./DANE/train_set/umowa_na_odleglosc_odstapieni...,0.148148,0.021621,0,0,0.750000
10846,197,./DANE/train_set/umowa_na_odleglosc_odstapieni...,0.065574,0.018559,0,0,1.000000
10847,198,./DANE/train_set/umowa_na_odleglosc_odstapieni...,0.121739,0.014643,0,0,1.000000


In [161]:
combined_df_metr['File'] = combined_df_metr['filepath'].apply(lambda x : x.split("\\")[-1].split('.')[0] )
combined_df_metr1 = combined_df_metr.drop(columns=['Unnamed: 0','filepath'])

In [162]:
combined_df_metr1

,numberEmpty,nonWhiteFraction,possibleShapes,possibleImages,nonEmptySections,File
0,0.146341,0.575719,4,2,0.500000,00133f89-017d-44f5-a549-e11cf5b5221a
1,0.161616,0.722688,2,1,0.500000,00a3e900-6616-46fe-a528-a5ac51952562
2,0.210526,0.086930,5,5,0.333333,0116aef3-5834-4a14-8dc0-e7eb3d79f5de
3,0.133333,0.814986,1,1,0.500000,024b107d-cd7f-4f81-a28c-50a9df08d586
4,0.142857,0.324584,2,2,0.500000,0296a9e1-5bec-456f-9e9c-fc8849203056
...,...,...,...,...,...,...
10844,0.187500,0.017731,0,0,0.500000,b70bbd24-3389-48a6-844b-6a3b20457e04
10845,0.148148,0.021621,0,0,0.750000,ba63205f-5084-4bbd-a620-4107ee2b6fad
10846,0.065574,0.018559,0,0,1.000000,bb6ebdb4-fb65-4b76-8a78-ea4a17ba942f
10847,0.121739,0.014643,0,0,1.000000,bc0f55f6-35a4-4de7-85e7-c2ff98b0efcc


In [164]:
df3  = df2.merge(combined_df_metr1,on='File')
df3_val  = df2_val.merge(combined_df_metr1,on='File')

In [167]:
y_train = df3["ID"]
X_train = df3.drop(columns=["File","Text ING", "ID", "Text Adam pl",'Nrow'])

y_test = df3_val["ID"]
X_val = df3_val.drop(columns=["File","Text ING", "ID", "Text Adam pl",'Nrow'])

In [168]:
len(pd.unique(y_train))

16

In [169]:
X_train

,isOther,string_length,capital_letters_count,numbers_count,question_marks_count,exclamation_marks_count,currency_signs_count,flag_phone,flag_text,original message,...,curriculum vitae,biographical sketch,redacted material,smoking,cigarettes,numberEmpty,nonWhiteFraction,possibleShapes,possibleImages,nonEmptySections
0,0,1723,0.268561,18,1,0,0,0,0,0,...,0,0,0,1,0,0.063953,0.407737,10,10,1.000000
1,0,2245,0.141140,37,1,0,0,0,0,0,...,0,0,0,0,0,0.017199,0.460661,1,1,0.750000
2,0,488,0.359918,29,0,0,0,0,0,0,...,0,0,0,0,0,0.181818,0.183766,8,4,1.000000
3,0,231,0.129310,0,0,0,0,0,0,0,...,0,0,0,0,0,0.043478,0.058376,0,0,0.666667
4,0,699,0.334286,7,0,0,0,0,0,0,...,0,0,0,4,0,0.108974,0.297160,4,3,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8070,0,302,0.059406,16,0,0,0,0,0,0,...,0,0,0,0,0,0.081395,0.016436,1,1,0.750000
8071,0,846,0.069658,22,0,0,0,0,0,0,...,0,0,0,0,0,0.071429,0.048382,2,2,1.000000
8072,0,1259,0.185714,63,0,0,3,1,0,0,...,0,0,0,0,0,0.079861,0.036614,0,0,0.666667
8073,0,2069,0.124155,39,1,1,1,0,0,0,...,0,0,0,0,0,0.016129,0.072717,1,1,1.000000


In [170]:
X_val

,isOther,string_length,capital_letters_count,numbers_count,question_marks_count,exclamation_marks_count,currency_signs_count,flag_phone,flag_text,original message,...,curriculum vitae,biographical sketch,redacted material,smoking,cigarettes,numberEmpty,nonWhiteFraction,possibleShapes,possibleImages,nonEmptySections
0,1,1371,0.450437,48,0,0,2,0,0,0,...,0,0,0,0,0,0.148515,0.272017,4,4,0.666667
1,0,622,0.065811,2,0,0,0,0,0,0,...,0,0,0,0,0,0.088608,0.067392,4,2,0.666667
2,1,1095,0.470803,26,0,0,2,0,0,0,...,0,0,0,0,0,0.222222,0.506489,8,8,0.750000
3,0,478,0.304802,20,0,0,0,0,0,0,...,0,0,0,0,0,0.200000,0.837641,2,2,0.666667
4,0,1912,0.192891,21,0,0,0,0,0,0,...,0,0,0,0,1,0.038328,0.222791,2,1,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,0,1508,0.145129,58,0,0,0,0,0,0,...,0,0,0,0,0,0.056548,0.072587,2,2,1.000000
903,0,1497,0.140854,50,0,0,1,0,0,0,...,0,0,0,0,0,0.034602,0.053375,1,1,0.750000
904,0,1403,0.184473,133,0,2,2,1,0,0,...,0,0,0,0,0,0.048276,0.073265,1,1,1.000000
905,0,1164,0.095279,48,0,0,0,0,0,0,...,0,0,0,0,0,0.062016,0.043456,1,0,1.000000


In [216]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_test)

params = {
    'objective': 'multi:softmax',
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'max_depth': 7,
    'learning_rate':0.04, 
    'booster':'gbtree',
    'num_class' : 16
}

model = xgb.train(params, dtrain, num_boost_round=700, evals=[(dval, 'validation')],
                  early_stopping_rounds=5)

y_pred = model.predict(dval)

C:\Users\szymo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[0]	validation-mlogloss:2.68705
[1]	validation-mlogloss:2.61352
[2]	validation-mlogloss:2.55106
[3]	validation-mlogloss:2.49558
[4]	validation-mlogloss:2.44770
[5]	validation-mlogloss:2.40322
[6]	validation-mlogloss:2.36252
[7]	validation-mlogloss:2.32577
[8]	validation-mlogloss:2.29156
[9]	validation-mlogloss:2.25981
[10]	validation-mlogloss:2.22945
[11]	validation-mlogloss:2.20091
[12]	validation-mlogloss:2.17517
[13]	validation-mlogloss:2.15052
[14]	validation-mlogloss:2.12674
[15]	validation-mlogloss:2.10441
[16]	validation-mlogloss:2.08383
[17]	validation-mlogloss:2.06376
[18]	validation-mlogloss:2.04503
[19]	validation-mlogloss:2.02667
[20]	validation-mlogloss:2.00929
[21]	validation-mlogloss:1.99293
[22]	validation-mlogloss:1.97724
[23]	validation-mlogloss:1.96227
[24]	validation-mlogloss:1.94842
[25]	validation-mlogloss:1.93519
[26]	validation-mlogloss:1.92159
[27]	validation-mlogloss:1.90941
[28]	validation-mlogloss:1.89699
[29]	validation-mlogloss:1.88590
[30]	validation-mlog

In [217]:
d = xgb.DMatrix(X_train)

y_pred_val = model.predict(d)

print('Train:')
print(f'Accuracy Score: {accuracy_score(y_train,y_pred_val)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_train, y_pred_val)}')
print(f'Recall score: {recall_score(y_train,y_pred_val, average="micro")}')
print(f"Precision score: {precision_score(y_train,y_pred_val, average='micro')}")

Train:
Accuracy Score: 0.9025386996904025
Confusion Matrix: 
[[484   3   1  10   1   5   1   1   0   2   5   2   1   0   1   1]
 [  9 445   0   5   4   7   6   1   0   3   5   0   0   1   5   3]
 [  1   3 486   0   2   2   4   0   0   2   2   0   2   2   0   0]
 [  0   0   0 505   0   0   1   0   0   0   0   0   0   0   0   0]
 [ 10   3   1   4 430  14  21   2   3   4   3   5   0   1   5   3]
 [ 10   2   0   7   1 512   7   1   1   4   2   2   0   0   1   1]
 [  7   5   0   2   5   8 460   1   0   1   3   1   0   0   2   1]
 [  4   1   2   2   6  11   4 438   6   3   6   4   2   3   2   1]
 [  4   1   4   2   2   8   2   3 451   2   7   2   7   2   4   1]
 [ 13   0   0   3   2  13   4   0   0 397   6   4   0   3   2   0]
 [ 18   3   1  14   4  13   1   2   5   7 447   2   1   4   8   1]
 [  6   4   0   4   3  17   8   2   5   8   3 419   4   1   7   4]
 [  2   0   1   2   1   6   1   3   3   2   5   0 479   2   1   0]
 [  3   3   1   2   0   8   1   5   0   4   5   1   1 482   2   1]
 

In [218]:
d = xgb.DMatrix(X_val)

y_pred_val = model.predict(d)

print('Validate:')
print(f'Accuracy Score: {accuracy_score(y_test,y_pred_val)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred_val)}')
print(f'Recall score: {recall_score(y_test,y_pred_val, average="micro")}')
print(f"Precision score: {precision_score(y_test,y_pred_val, average='micro')}")

Validate:
Accuracy Score: 0.5611907386990077
Confusion Matrix: 
[[31  0  0  7  4  5  2  1  0  5  2  0  0  0  1  0]
 [ 1 28  1  3  2  2 12  0  0  1  2  0  0  1  3  0]
 [ 1  3 42  1  1  1  0  3  2  2  1  0  0  0  0  0]
 [ 6  0  0 37  1  6  0  0  0  0  6  0  0  0  1  0]
 [ 4  3  1  2 23  6  4  2  3  0  1  1  1  3  1  2]
 [ 4  2  0  5  1 42  3  0  0  1  2  1  0  0  0  0]
 [ 1  9  0  2  1  3 31  0  2  1  0  0  0  0  2  4]
 [ 0  0  0  1  3  5  0 28  9  0  2  1  1  0  4  2]
 [ 0  1  0  1  2  1  3  3 37  0  2  0  1  1  3  1]
 [ 1  1  0  0  0  4  1  0  1 22  2  3  2 12  2  0]
 [ 3  4  1  4  3  4  1  0  1  3 29  0  0  3  3  0]
 [ 3  1  0  2  4 10  2  3  0  0  1 25  2  1  0  2]
 [ 0  1  1  0  0  0  0  2  2  0  2  0 45  2  2  0]
 [ 3  0  1  1  0  1  0  0  1  6  1  0  2 40  1  1]
 [ 2  4  2  1  7  6  2  1  4  2  2  3  2  1 18  2]
 [ 0  1  0  0  9  3  3  0  0  1  0  0  3  1  1 31]]
Recall score: 0.5611907386990077
Precision score: 0.5611907386990077


C:\Users\szymo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [214]:
model.save_model("xgboost_model_new.json")

In [215]:
model.get_score(importance_type='gain')

{'isOther': 2.013657808303833,
 'string_length': 2.9583899974823,
 'capital_letters_count': 2.1419310569763184,
 'numbers_count': 2.5259876251220703,
 'question_marks_count': 5.765464782714844,
 'exclamation_marks_count': 1.0143450498580933,
 'currency_signs_count': 1.4487671852111816,
 'flag_phone': 2.5147199630737305,
 'cc:': 3.1309030055999756,
 'subject:': 12.428869247436523,
 're:': 1.5530774593353271,
 'attachments': 0.8042263984680176,
 'dear': 13.851824760437012,
 'regards': 2.4135985374450684,
 'sincerely': 6.531022548675537,
 'miss': 2.361471176147461,
 'sir': 1.1539700031280518,
 'invoice': 7.015165328979492,
 'voucher': 3.320788860321045,
 'amount': 1.5712310075759888,
 'description': 1.3633835315704346,
 'estimate': 1.572384238243103,
 'rate': 1.1555410623550415,
 'payment': 2.502699375152588,
 'price': 1.2536025047302246,
 'form': 2.0858283042907715,
 'request': 1.7399078607559204,
 'report': 2.814318895339966,
 'confidential': 1.5732675790786743,
 'memo': 14.785046577453